In [8]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

%pip install numpy-financial yfinance pandas numpy matplotlib --quiet
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime




Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Group Assignment
### Team Number: 18
### Team Member Names: Poneesh Kumar, Ian Leung, Vivaan Goel
### Team Strategy Chosen: Market Beat(Market Beat, Market Meet, Risk-Free)

In [9]:
filename = "Tickers_Example.csv"
tickers = pd.read_csv(filename, header=None, names=['Ticker'])
tickers.head()

,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Poneesh, Ian, Vivaan

In [10]:

# temp
# beta, volume, regularMarketVolume, averageVolume, average(Daily)Volume10days/Day, marketCap, 
# currency, earningsQuarterlyGrowth, financialCurrency, market = us_market

history = {}
def averageDailyVolume(ticker):
    start_date = "2024-10-01"
    end_date = "2025-09-30"

    history[ticker.ticker] = ticker.history(start=start_date, end=end_date)

    daily_volume = history[ticker.ticker]['Volume']

    daily_volume.index = daily_volume.index.tz_localize(None)

    volume_df = daily_volume.to_frame()
    volume_df['Months'] = volume_df.index.to_period('M')

    daily_volume.index = pd.to_datetime(daily_volume.index)
    daily_volume.index = daily_volume.index.strftime('%Y-%m-%d') 
    volume_df.index = pd.to_datetime(volume_df.index)
    volume_df.index = volume_df.index.strftime('%Y-%m-%d') 

    monthly_counts = volume_df.groupby('Months')['Volume'].count()
    valid_months = monthly_counts[monthly_counts >= 18].index
    valid_vol_df = volume_df[volume_df['Months'].isin(valid_months)]

    volumes = []
    
    for volume in valid_vol_df['Volume']:
        volumes.append(volume)
    
    average_daily_vol = np.mean(volumes)
    return average_daily_vol
    



In [11]:
def scoring1(ticker):
    start_date = "2025-11-01"
    end_date = "2025-11-14"

    history[ticker.ticker] = ticker.history(start=start_date, end=end_date)

    daily_close = history[ticker.ticker]['Close']
    daily_volume = history[ticker.ticker]['Volume']

    daily_close.index = daily_close.index.tz_localize(None)
    daily_volume.index = daily_volume.index.tz_localize(None)

    daily_close.index = pd.to_datetime(daily_close.index)
    daily_close.index = daily_close.index.strftime('%Y-%m-%d') 
    daily_volume.index = pd.to_datetime(daily_volume.index)
    daily_volume.index = daily_volume.index.strftime('%Y-%m-%d') 
    
    price_change = daily_close.pct_change()
    price_change.drop(index=price_change.index[0], inplace=True)
    volume_change = daily_volume.pct_change()
    volume_change.drop(index=volume_change.index[0], inplace=True)
    
    avg_price_roc = np.mean(price_change)*100
    avg_vol_roc = np.mean(volume_change)*100

    # Calculating the Relative Strength Index
    delta = daily_close.astype(float).diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    window = 7
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    latest_rsi = rsi.iloc[-1]

    # Calculating the simple moving average
    sma_7 = daily_close.astype(float).rolling(window=7).mean().iloc[-1]

    #Calculating the last closing price
    last_close = daily_close.iloc[-1].round(2)

    # Find the difference between the SMA_7 and last closing price as a %
    sma_trend_strength = ((last_close - sma_7)/ last_close) * 100
    

    return avg_price_roc, avg_vol_roc, latest_rsi, sma_7, sma_trend_strength

In [12]:
tickers_df = pd.DataFrame(columns=[
    'Ticker', 'Sector', 'Market Cap', 'Country', 'Market',
    'Avg Daily Vol', 'Beta',
    'Price % ROC', 'Vol % ROC',
    'RSI', 'SMA', 'SMA Trend'
])

row = 0
for ticker_sym in tickers['Ticker']:
    ticker = yf.Ticker(ticker_sym)
    try:
        price_roc, vol_roc, rsi_val, sma_val, sma_trend = scoring1(ticker)
        tickers_df.loc[row] = {
        'Ticker': ticker_sym,
        'Sector': ticker.info['sector'],
        'Market Cap': ticker.info['marketCap'],
        'Country': ticker.info['country'],
        'Market': ticker.info['market'],
        'Avg Daily Vol': averageDailyVolume(ticker),
        'Beta': ticker.info['beta'],
        'Price % ROC': price_roc.round(3),
        'Vol % ROC': vol_roc.round(3),
        'RSI': rsi_val.round(3),
        'SMA': sma_val.round(3),
        'SMA Trend': sma_trend.round(3)
        }

        row+= 1
    except:
        print(f"{ticker_sym} invalid, skipping...")

tickers_df

$AGN: possibly delisted; no timezone found


AGN invalid, skipping...


$CELG: possibly delisted; no timezone found


CELG invalid, skipping...


$MON: possibly delisted; no timezone found


MON invalid, skipping...


$RTN: possibly delisted; no timezone found


RTN invalid, skipping...


,Ticker,Sector,Market Cap,Country,Market,Avg Daily Vol,Beta,Price % ROC,Vol % ROC,RSI,SMA,SMA Trend
0,AAPL,Technology,3951253782528,United States,us_market,5.384430e+07,1.109,0.196,0.370,64.276,271.242,0.626
1,ABBV,Healthcare,405526380544,United States,us_market,6.362559e+06,0.352,1.161,3.614,92.753,223.481,3.792
2,ABT,Healthcare,215764713472,United States,us_market,6.189108e+06,0.719,0.540,-0.376,75.000,126.689,1.989
3,ACN,Technology,149976694784,Ireland,us_market,3.444719e+06,1.262,-0.029,0.249,59.717,245.161,0.973
4,AIG,Financial Services,41932550144,United States,us_market,4.416646e+06,0.622,-0.070,5.382,35.360,76.993,1.732
5,AMZN,Consumer Cyclical,2321273585664,United States,us_market,4.246964e+07,1.368,-0.819,-5.634,27.088,245.276,-3.239
6,AXP,Financial Services,239244312576,United States,us_market,2.800084e+06,1.147,0.112,4.785,59.734,367.944,-0.881
7,BA,Industrials,136345935872,United States,us_market,9.557990e+06,1.170,-0.617,-0.258,16.952,195.547,-0.497
8,BAC,Financial Services,377754320896,United States,us_market,3.933537e+07,1.304,-0.154,8.268,41.966,53.281,-0.778
9,BB.TO,Technology,3472060416,Canada,ca_market,2.494208e+06,1.023,-1.242,0.666,24.444,6.404,-4.988


In [13]:
filtered_df = tickers_df[(tickers_df["Country"] == 'Canada') | (tickers_df["Country"] == "United States")].reset_index(drop=True)
filtered_df = filtered_df[(filtered_df["Market"] == 'ca_market') | (filtered_df["Market"] == "us_market")].reset_index(drop=True)
filtered_df = filtered_df[filtered_df["Avg Daily Vol"] > 5000].reset_index(drop=True)
filtered_df

,Ticker,Sector,Market Cap,Country,Market,Avg Daily Vol,Beta,Price % ROC,Vol % ROC,RSI,SMA,SMA Trend
0,AAPL,Technology,3951253782528,United States,us_market,5.384430e+07,1.109,0.196,0.370,64.276,271.242,0.626
1,ABBV,Healthcare,405526380544,United States,us_market,6.362559e+06,0.352,1.161,3.614,92.753,223.481,3.792
2,ABT,Healthcare,215764713472,United States,us_market,6.189108e+06,0.719,0.540,-0.376,75.000,126.689,1.989
3,AIG,Financial Services,41932550144,United States,us_market,4.416646e+06,0.622,-0.070,5.382,35.360,76.993,1.732
4,AMZN,Consumer Cyclical,2321273585664,United States,us_market,4.246964e+07,1.368,-0.819,-5.634,27.088,245.276,-3.239
5,AXP,Financial Services,239244312576,United States,us_market,2.800084e+06,1.147,0.112,4.785,59.734,367.944,-0.881
6,BA,Industrials,136345935872,United States,us_market,9.557990e+06,1.170,-0.617,-0.258,16.952,195.547,-0.497
7,BAC,Financial Services,377754320896,United States,us_market,3.933537e+07,1.304,-0.154,8.268,41.966,53.281,-0.778
8,BB.TO,Technology,3472060416,Canada,ca_market,2.494208e+06,1.023,-1.242,0.666,24.444,6.404,-4.988
9,BIIB,Healthcare,24672387072,United States,us_market,1.555065e+06,0.145,1.067,7.369,93.113,158.244,3.937


In [14]:
def get_score(ticker_df):
    ticker_df['Score'] = (
        # 1: Momentum (75%)
        350 * bounded_score(ticker_df['Price % ROC'], 0, 1.2) +
        250 * bounded_score(ticker_df['RSI'], 40, 80) +
        150 * bounded_score(ticker_df['SMA Trend'], 0.5, -2) + 
        
        # 2: Volume (20%)
        200 * bounded_score(ticker_df['Vol % ROC'], 0, 7) + 

        # 3: Volatility (5%)
        200 * bounded_score(ticker_df['Beta'], 0.5, 1.5)
        )

def bounded_score(col, min, max):
    return np.round(np.clip((col - min) / (max - min), 0, 1), 3)

In [15]:
get_score(filtered_df)
filtered_df = filtered_df.sort_values(by='Score', ascending=False).reset_index(drop=True)
filtered_df

,Ticker,Sector,Market Cap,Country,Market,Avg Daily Vol,Beta,Price % ROC,Vol % ROC,RSI,SMA,SMA Trend,Score
0,MRK,Healthcare,237214482432,United States,us_market,1.396483e+07,0.319,1.510,6.055,100.000,88.361,4.916,773.00
1,BIIB,Healthcare,24672387072,United States,us_market,1.555065e+06,0.145,1.067,7.369,93.113,158.244,3.937,761.15
2,ABBV,Healthcare,405526380544,United States,us_market,6.362559e+06,0.352,1.161,3.614,92.753,223.481,3.792,692.00
3,CAT,Industrials,256853114880,United States,us_market,2.586073e+06,1.562,-0.349,9.109,54.656,566.769,-2.388,641.50
4,LLY,Healthcare,935264387072,United States,us_market,3.976943e+06,0.389,1.675,-0.399,90.805,967.655,5.259,600.00
5,PM,Consumer Defensive,242290802688,United States,us_market,5.737421e+06,0.444,0.889,3.098,84.708,153.467,1.085,597.95
6,SHOP.TO,Technology,265471803392,Canada,ca_market,2.102246e+06,2.823,-1.988,7.737,26.346,219.421,-6.577,550.00
7,PFE,Healthcare,138731274240,United States,us_market,4.455903e+07,0.425,0.795,1.288,94.420,24.942,3.288,518.85
8,BK,Financial Services,75079942144,United States,us_market,3.981636e+06,1.123,0.344,3.094,72.419,110.377,0.597,515.95
9,AXP,Financial Services,239244312576,United States,us_market,2.800084e+06,1.147,0.112,4.785,59.734,367.944,-0.881,504.80


In [ ]:
def find_tier(tiers):
    # return the first tier index where the count (tier[0]) is > 0
    for i, tier in enumerate(tiers):
        if tier[0] > 0:
            return i
    # fallback if no tier met the condition
    return False

def find_percent_tier(percent):
    # simple allocation policy for additional tickers
    if percent <= 5:
        return 2
    elif percent <= 10:
        return 1
    return 0

def make_portfolio(df):
    portfolio = {}
    industry_set = {}

    # Tiers:
    # 4 stocks at 15%
    # 2 stocks at 10%
    # 4 stocks at 5%
    tiers = [(4, 15), (2, 10), (4, 5)]

    for _, row in df.iterrows():

        # Find available tiers, break when portfolio is full
        tier = find_tier(tiers)
        if tier is False:
            break
        percent = tiers[tier][1]

        # Store industry percentages
        if row["Sector"] not in industry_set:
            portfolio[row["Ticker"]] = percent
            industry_set[row["Sector"]] = percent
            tiers[tier] = (tiers[tier][0] - 1, tiers[tier][1])

        else:
            industry_cap = 40 - industry_set[row["Sector"]]
            if industry_cap <= 0:
                    continue
            if percent > industry_cap:
                percent = industry_cap
                tier = find_percent_tier(percent)
            portfolio[row["Ticker"]] = percent
            industry_set[row["Sector"]] += percent
            tiers[tier] = (tiers[tier][0] - 1, tiers[tier][1])


    return portfolio, industry_set
        

In [38]:
portfolio, industries = make_portfolio(filtered_df)
portfolio

{'MRK': 15,
 'BIIB': 15,
 'ABBV': 10,
 'CAT': 15,
 'PM': 15,
 'SHOP.TO': 10,
 'BK': 5,
 'AXP': 5,
 'KO': 5,
 'T.TO': 5}

In [39]:
industries

{'Healthcare': 40,
 'Industrials': 15,
 'Consumer Defensive': 20,
 'Technology': 10,
 'Financial Services': 10,
 'Communication Services': 5}